In [1]:
def count_train_test_val(count_val):
    count=[]
    count.append(count_val)
    count.append(round((0.7*count_val),0))
    count.append(round((0.2*count_val),0))
    count.append(round((0.1*count_val),0))
    
    total_add=count[1]+count[2]+count[3]
    tbs=total_add-count_val
    
    if (tbs!=0):
        count[1]=count[1]-tbs
    if (count[3]==0):
        count[1]=count[1]-1
        count[3]=1
    
    count = map(int, count)
    return count
      

In [2]:
def saving_images_Train_Test_Val(Excelname,Viewplane,save_location,setnumber):
    import pandas as pd
    import scipy.io
    import matlab.engine
    eng = matlab.engine.start_matlab()
    import sys
    sys.path.append('/data/Gurpreet/CODE/VC2/OCR')
    from scipy import misc
    from IPython.display import clear_output
    
    excelfilename="/data/Gurpreet/Echo/Excel_Files/"+Excelname
    temp_excel=pd.read_excel(excelfilename, sheetname=Viewplane, header=0)
    temp_excel= temp_excel.sample(frac=1).reset_index(drop=True)
    count_temp=count_train_test_val(len(temp_excel))
    
    Train_Excel=temp_excel.iloc[0:count_temp[1]]
    Test_Excel=temp_excel.iloc[count_temp[1]:(count_temp[1]+count_temp[2])]
    Val_Excel=temp_excel.iloc[(count_temp[1]+count_temp[2]):(count_temp[1]+count_temp[2]+count_temp[3])]
    
    writexcel=str(save_location)+str(setnumber)+"/Dataset"+str(Viewplane)+".xls"
    writer = pd.ExcelWriter(writexcel)
    Train_sheetname="Train_"+Viewplane
    Test_sheetname="Test_"+Viewplane
    Val_sheetname="Val_"+Viewplane
    
    Train_Excel.to_excel(writer,Train_sheetname)
    Test_Excel.to_excel(writer,Test_sheetname)
    Val_Excel.to_excel(writer,Val_sheetname)
    
    writer.save()
    
    #saving Train
    for i in xrange(0,count_temp[1]):
        patnumber=temp_excel.iloc[i,0]
        Videonumber=temp_excel.iloc[i,1]
        Viewplane=temp_excel.iloc[i,3]
        imgname="/data/Gurpreet/Echo/"+str(patnumber)+"/"+str(patnumber)+"("+str(Videonumber)+").dcm"
        [MANUFACTURER,WIDTH,HEIGHT]=eng.get_file_and_info3(imgname,220,nargout=3)
        tempfile="/home/gus2011/tempimage_220.mat"
        IMG=scipy.io.loadmat(tempfile, mat_dtype=True)['IMG']
        SoI=IMG.shape
        Totalframes=SoI[3]
        print imgname+"    "+Viewplane
        for j in xrange(0,Totalframes):
            IMG2=IMG[:,:,:,j]
            k=j+1
            savename=str(save_location)+str(setnumber)+"/dataset/train/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            IM=IMG2
            #IM=GE(IMG2,MANUFACTURER,WIDTH,HEIGHT,0)
            misc.imsave(savename,IM)
            
    #saving Test
    for i in xrange(count_temp[1],(count_temp[1]+count_temp[2])):
        patnumber=temp_excel.iloc[i,0]
        Videonumber=temp_excel.iloc[i,1]
        Viewplane=temp_excel.iloc[i,3]
        imgname="/data/Gurpreet/Echo/"+str(patnumber)+"/"+str(patnumber)+"("+str(Videonumber)+").dcm"
        [MANUFACTURER,WIDTH,HEIGHT]=eng.get_file_and_info3(imgname,220,nargout=3)
        tempfile="/home/gus2011/tempimage_220.mat"
        IMG=scipy.io.loadmat(tempfile, mat_dtype=True)['IMG']
        SoI=IMG.shape
        Totalframes=SoI[3]
        print imgname+"    "+Viewplane
        for j in xrange(0,Totalframes):
            IMG2=IMG[:,:,:,j]
            k=j+1
            savename=str(save_location)+str(setnumber)+"/dataset/test/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            IM=IMG2
            #IM=GE(IMG2,MANUFACTURER,WIDTH,HEIGHT,0)
            misc.imsave(savename,IM)
            
    #saving val
    for i in xrange((count_temp[1]+count_temp[2]),(count_temp[1]+count_temp[2]+count_temp[3])):
        patnumber=temp_excel.iloc[i,0]
        Videonumber=temp_excel.iloc[i,1]
        Viewplane=temp_excel.iloc[i,3]
        imgname="/data/Gurpreet/Echo/"+str(patnumber)+"/"+str(patnumber)+"("+str(Videonumber)+").dcm"
        [MANUFACTURER,WIDTH,HEIGHT]=eng.get_file_and_info3(imgname,220,nargout=3)
        tempfile="/home/gus2011/tempimage_220.mat"
        IMG=scipy.io.loadmat(tempfile, mat_dtype=True)['IMG']
        SoI=IMG.shape
        Totalframes=SoI[3]
        print imgname+"    "+Viewplane
        for j in xrange(0,Totalframes):
            IMG2=IMG[:,:,:,j]
            k=j+1
            savename=str(save_location)+str(setnumber)+"/dataset/val/"+str(Viewplane)+"/"+str(patnumber)+"_"+str(Videonumber)+"_"+str(k)+".jpg"
            IM=IMG2
            #IM=GE(IMG2,MANUFACTURER,WIDTH,HEIGHT,0)
            misc.imsave(savename,IM)
    clear_output()

In [3]:
Viewplanes=['A2C','A3C','A4C','PLA','PSA','SSX','SCX']
for j in xrange(0,10):
    k=j+1
    setnumber="SET"+str(k)
    for i in xrange(0,7):
        Viewplane=Viewplanes[i]
        saving_images_Train_Test_Val("View_point_27062017.xlsx",Viewplane,"/data/Gurpreet/RUNS/VC_10/",setnumber)
        

In [ ]:
datapath="/data/Gurpreet/RUNS/VC_10/"+setnumber+"/dataset/"
savepath="/data/Gurpreet/RUNS/VC_10/"+setnumber+"/ResNetModel.pth.tar"

res = models.resnet18(pretrained=True)
res.fc = nn.Linear(res.fc.in_features,3)
obj = model_pip(model_in=res,scale=True,batch_size=30,use_gpu=True,gpu=3,data_path=datapath)
model = obj.train_model(epochs=10)
obj.store_model(f_name=savepath)